In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import VBox, HBox, IntSlider, FloatSlider, Output, interactive

# Load the grayscale image (uncolored)
image_path = "images.jpeg"  # Replace with the actual image path
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if image is None:
    raise FileNotFoundError(f"Image at path '{image_path}' could not be found or loaded.")
else:
    print("Image pixel values range:", image.min(), "to", image.max())


krypton_laser_colors = {
    406: [138, 43, 226],    # Violet
    413: [75, 0, 130],      # Indigo
    420: [0, 0, 255],       # Blue
    530: [0, 255, 0],       # Green
    568: [255, 255, 0],     # Yellow
    600: [255, 165, 0],     # Orange
    647: [255, 0, 0],       # Red
}

argon_laser_colors = {
    457.9: [0, 0, 255],     # Blue
    465.8: [75, 0, 130],    # Indigo
    476.5: [0, 255, 255],   # Cyan
    488: [0, 255, 0],       # Green
    514.5: [0, 128, 0],     # Dark Green
    540: [255, 255, 0],     # Yellow
    600: [255, 165, 0],     # Orange
}

# Function to map wavelength to colors based on laser type
def get_color_from_wavelength(wavelength, laser_type):
    laser_colors = krypton_laser_colors if laser_type == "krypton" else argon_laser_colors
    # Get nearest available color for smoother transitions
    closest_wavelength = min(laser_colors.keys(), key=lambda k: abs(k - wavelength))
    return laser_colors[closest_wavelength]

# Function to apply colors based on intensity ranges and laser wavelengths
def apply_multicolor_with_lasers(image, wavelengths, intensity_ranges, laser_type="krypton"):
    colored_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    for idx, (lower, upper) in enumerate(intensity_ranges):
        color = get_color_from_wavelength(wavelengths[idx], laser_type)
        mask = (image >= lower) & (image <= upper)
        for i in range(3):
            colored_image[:, :, i] = np.where(mask, color[i], colored_image[:, :, i])
    return colored_image

# Function to display images side by side
def update_images(
    k_w1, k_w2, k_w3, k_w4, k_r1_lower, k_r1_upper, k_r2_lower, k_r2_upper, k_r3_lower, k_r3_upper, k_r4_lower, k_r4_upper,
    a_w1, a_w2, a_w3, a_w4, a_r1_lower, a_r1_upper, a_r2_lower, a_r2_upper, a_r3_lower, a_r3_upper, a_r4_lower, a_r4_upper
):
    # Krypton image
    krypton_wavelengths = [k_w1, k_w2, k_w3, k_w4]
    krypton_ranges = [
        (k_r1_lower, k_r1_upper),
        (k_r2_lower, k_r2_upper),
        (k_r3_lower, k_r3_upper),
        (k_r4_lower, k_r4_upper),
    ]
    krypton_image = apply_multicolor_with_lasers(image, krypton_wavelengths, krypton_ranges, "krypton")

    # Argon image
    argon_wavelengths = [a_w1, a_w2, a_w3, a_w4]
    argon_ranges = [
        (a_r1_lower, a_r1_upper),
        (a_r2_lower, a_r2_upper),
        (a_r3_lower, a_r3_upper),
        (a_r4_lower, a_r4_upper),
    ]
    argon_image = apply_multicolor_with_lasers(image, argon_wavelengths, argon_ranges, "argon")

    # Display side-by-side images
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(krypton_image)
    axs[0].set_title(f'Krypton Laser Image\nWavelengths: {krypton_wavelengths}')
    axs[0].axis('off')
    axs[1].imshow(argon_image)
    axs[1].set_title(f'Argon Laser Image\nWavelengths: {argon_wavelengths}')
    axs[1].axis('off')
    plt.show()

# Krypton sliders with additional wavelength and range controls
krypton_sliders = VBox([
    FloatSlider(min=400, max=700, step=1, value=406, description='Krypton Wavelength 1'),
    FloatSlider(min=400, max=700, step=1, value=413, description='Krypton Wavelength 2'),
    FloatSlider(min=400, max=700, step=1, value=530, description='Krypton Wavelength 3'),
    FloatSlider(min=400, max=700, step=1, value=647, description='Krypton Wavelength 4'),
    IntSlider(min=0, max=255, step=1, value=0, description='Range 1 Lower'),
    IntSlider(min=0, max=255, step=1, value=64, description='Range 1 Upper'),
    IntSlider(min=0, max=255, step=1, value=65, description='Range 2 Lower'),
    IntSlider(min=0, max=255, step=1, value=128, description='Range 2 Upper'),
    IntSlider(min=0, max=255, step=1, value=129, description='Range 3 Lower'),
    IntSlider(min=0, max=255, step=1, value=192, description='Range 3 Upper'),
    IntSlider(min=0, max=255, step=1, value=193, description='Range 4 Lower'),
    IntSlider(min=0, max=255, step=1, value=255, description='Range 4 Upper'),
])

# Argon sliders with additional wavelength and range controls
argon_sliders = VBox([
    FloatSlider(min=400, max=700, step=1, value=457.9, description='Argon Wavelength 1'),
    FloatSlider(min=400, max=700, step=1, value=476.5, description='Argon Wavelength 2'),
    FloatSlider(min=400, max=700, step=1, value=514.5, description='Argon Wavelength 3'),
    FloatSlider(min=400, max=700, step=1, value=600, description='Argon Wavelength 4'),
    IntSlider(min=0, max=255, step=1, value=0, description='Range 1 Lower'),
    IntSlider(min=0, max=255, step=1, value=64, description='Range 1 Upper'),
    IntSlider(min=0, max=255, step=1, value=65, description='Range 2 Lower'),
    IntSlider(min=0, max=255, step=1, value=128, description='Range 2 Upper'),
    IntSlider(min=0, max=255, step=1, value=129, description='Range 3 Lower'),
    IntSlider(min=0, max=255, step=1, value=192, description='Range 3 Upper'),
    IntSlider(min=0, max=255, step=1, value=193, description='Range 4 Lower'),
    IntSlider(min=0, max=255, step=1, value=255, description='Range 4 Upper'),
])

# Link sliders to the interactive function
interactive_plot = interactive(
    update_images,
    k_w1=krypton_sliders.children[0],
    k_w2=krypton_sliders.children[1],
    k_w3=krypton_sliders.children[2],
    k_w4=krypton_sliders.children[3],
    k_r1_lower=krypton_sliders.children[4],
    k_r1_upper=krypton_sliders.children[5],
    k_r2_lower=krypton_sliders.children[6],
    k_r2_upper=krypton_sliders.children[7],
    k_r3_lower=krypton_sliders.children[8],
    k_r3_upper=krypton_sliders.children[9],
    k_r4_lower=krypton_sliders.children[10],
    k_r4_upper=krypton_sliders.children[11],
    a_w1=argon_sliders.children[0],
    a_w2=argon_sliders.children[1],
    a_w3=argon_sliders.children[2],
    a_w4=argon_sliders.children[3],
    a_r1_lower=argon_sliders.children[4],
    a_r1_upper=argon_sliders.children[5],
    a_r2_lower=argon_sliders.children[6],
    a_r2_upper=argon_sliders.children[7],
    a_r3_lower=argon_sliders.children[8],
    a_r3_upper=argon_sliders.children[9],
    a_r4_lower=argon_sliders.children[10],
    a_r4_upper=argon_sliders.children[11]
)

# Arrange Krypton and Argon sliders above their respective images
ui_layout = VBox([
    HBox([VBox([krypton_sliders]), VBox([argon_sliders])]),
    interactive_plot.children[-1],
])

display(ui_layout)

Image pixel values range: 0 to 248


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import VBox, HBox, FloatSlider, IntSlider, interactive_output

# Load the MRI image (grayscale)
image_path = "medical_scan.jpg"  # Replace with your actual image path
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if image is None:
    raise FileNotFoundError(f"Image at path '{image_path}' could not be found or loaded.")
else:
    print("Image pixel values range:", image.min(), "to", image.max())

# Laser color mappings
krypton_laser_colors = {
    406: [138, 43, 226],    # Violet
    413: [75, 0, 130],      # Indigo
    420: [0, 0, 255],       # Blue
    530: [0, 255, 0],       # Green
    568: [255, 255, 0],     # Yellow
    600: [255, 165, 0],     # Orange
    647: [255, 0, 0],       # Red
}

argon_laser_colors = {
    457.9: [0, 0, 255],     # Blue
    465.8: [75, 0, 130],    # Indigo
    476.5: [0, 255, 255],   # Cyan
    488: [0, 255, 0],       # Green
    514.5: [255, 255, 0],   # Yellow
    540: [255, 165, 0],     # Orange
    568: [255, 105, 180],   # Hot Pink
    600: [255, 69, 0],      # Red-Orange
    620: [255, 0, 0],       # Red
    700: [128, 0, 0],       # Maroon
}

# Function to apply laser color mapping
def apply_laser_coloring(image, wavelengths, ranges, laser_colors):
    colored_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    for i, wavelength in enumerate(wavelengths):
        lower, upper = ranges[i]
        mask = (image >= lower) & (image <= upper)
        color = laser_colors.get(wavelength, [255, 255, 255])  # Default: white
        for j in range(3):  # Apply color to R, G, B channels
            colored_image[:, :, j] = np.where(mask, color[j], colored_image[:, :, j])
    return colored_image

# Interactive update function
def update_images(
    k_w1, k_w2, k_w3, k_w4,
    k_r1_lower, k_r1_upper,
    k_r2_lower, k_r2_upper,
    k_r3_lower, k_r3_upper,
    k_r4_lower, k_r4_upper,
    a_w1, a_w2, a_w3, a_w4,
    a_r1_lower, a_r1_upper,
    a_r2_lower, a_r2_upper,
    a_r3_lower, a_r3_upper,
    a_r4_lower, a_r4_upper,
):
    # Krypton processing
    krypton_wavelengths = [k_w1, k_w2, k_w3, k_w4]
    krypton_ranges = [
        (k_r1_lower, k_r1_upper),
        (k_r2_lower, k_r2_upper),
        (k_r3_lower, k_r3_upper),
        (k_r4_lower, k_r4_upper),
    ]
    krypton_image = apply_laser_coloring(image, krypton_wavelengths, krypton_ranges, krypton_laser_colors)

    # Argon processing
    argon_wavelengths = [a_w1, a_w2, a_w3, a_w4]
    argon_ranges = [
        (a_r1_lower, a_r1_upper),
        (a_r2_lower, a_r2_upper),
        (a_r3_lower, a_r3_upper),
        (a_r4_lower, a_r4_upper),
    ]
    argon_image = apply_laser_coloring(image, argon_wavelengths, argon_ranges, argon_laser_colors)

    # Display images
    fig, axs = plt.subplots(1, 2, figsize=(14, 7))
    axs[0].imshow(krypton_image)
    axs[0].set_title("Krypton Laser Image")
    axs[0].axis('off')

    axs[1].imshow(argon_image)
    axs[1].set_title("Argon Laser Image")
    axs[1].axis('off')

    # Print tissue categories and colors to console
    tissue_info = [
        ("Gray Matter", "Indigo"),
        ("White Matter", "Green"),
        ("CSF (Cerebrospinal Fluid)", "Blue"),
        ("Vessels/Tumors", "Red")
    ]
    
    print("\nTissue Categories and Colors:")
    for tissue, color in tissue_info:
        print(f"{tissue}: {color}")

    plt.show()

# Krypton sliders
krypton_sliders = VBox([
    HBox([
        FloatSlider(min=400, max=700, step=0.1, value=406, description='W1'),
        FloatSlider(min=400, max=700, step=0.1, value=413, description='W2'),
        FloatSlider(min=400, max=700, step=0.1, value=530, description='W3'),
        FloatSlider(min=400, max=700, step=0.1, value=647, description='W4'),
    ]),
    HBox([
        IntSlider(min=0, max=255, step=1, value=0, description='R1 Low'),
        IntSlider(min=0, max=255, step=1, value=64, description='R1 High'),
    ]),
    HBox([
        IntSlider(min=0, max=255, step=1, value=65, description='R2 Low'),
        IntSlider(min=0, max=255, step=1, value=128, description='R2 High'),
    ]),
    HBox([
        IntSlider(min=0, max=255, step=1, value=129, description='R3 Low'),
        IntSlider(min=0, max=255, step=1, value=192, description='R3 High'),
    ]),
    HBox([
        IntSlider(min=0, max=255, step=1, value=193, description='R4 Low'),
        IntSlider(min=0, max=255, step=1, value=255, description='R4 High'),
    ]),
])

# Argon sliders
argon_sliders = VBox([
    HBox([
        FloatSlider(min=457.9, max=700, step=0.1, value=457.9, description='W1'),
        FloatSlider(min=457.9, max=700, step=0.1, value=465.8, description='W2'),
        FloatSlider(min=457.9, max=700, step=0.1, value=488, description='W3'),
        FloatSlider(min=457.9, max=700, step=0.1, value=514.5, description='W4'),
    ]),
    HBox([
        IntSlider(min=0, max=255, step=1, value=0, description='R1 Low'),
        IntSlider(min=0, max=255, step=1, value=64, description='R1 High'),
    ]),
    HBox([
        IntSlider(min=0, max=255, step=1, value=65, description='R2 Low'),
        IntSlider(min=0, max=255, step=1, value=128, description='R2 High'),
    ]),
    HBox([
        IntSlider(min=0, max=255, step=1, value=129, description='R3 Low'),
        IntSlider(min=0, max=255, step=1, value=192, description='R3 High'),
    ]),
    HBox([
        IntSlider(min=0, max=255, step=1, value=193, description='R4 Low'),
        IntSlider(min=0, max=255, step=1, value=255, description='R4 High'),
    ]),
])

# Interactive widgets
interactive_display = interactive_output(
    update_images,
    {
        'k_w1': krypton_sliders.children[0].children[0],
        'k_w2': krypton_sliders.children[0].children[1],
        'k_w3': krypton_sliders.children[0].children[2],
        'k_w4': krypton_sliders.children[0].children[3],
        'k_r1_lower': krypton_sliders.children[1].children[0],
        'k_r1_upper': krypton_sliders.children[1].children[1],
        'k_r2_lower': krypton_sliders.children[2].children[0],
        'k_r2_upper': krypton_sliders.children[2].children[1],
        'k_r3_lower': krypton_sliders.children[3].children[0],
        'k_r3_upper': krypton_sliders.children[3].children[1],
        'k_r4_lower': krypton_sliders.children[4].children[0],
        'k_r4_upper': krypton_sliders.children[4].children[1],
        'a_w1': argon_sliders.children[0].children[0],
        'a_w2': argon_sliders.children[0].children[1],
        'a_w3': argon_sliders.children[0].children[2],
        'a_w4': argon_sliders.children[0].children[3],
        'a_r1_lower': argon_sliders.children[1].children[0],
        'a_r1_upper': argon_sliders.children[1].children[1],
        'a_r2_lower': argon_sliders.children[2].children[0],
        'a_r2_upper': argon_sliders.children[2].children[1],
        'a_r3_lower': argon_sliders.children[3].children[0],
        'a_r3_upper': argon_sliders.children[3].children[1],
        'a_r4_lower': argon_sliders.children[4].children[0],
        'a_r4_upper': argon_sliders.children[4].children[1],
    }
)

# Display sliders
VBox([krypton_sliders, argon_sliders, interactive_display])


Image pixel values range: 0 to 255


In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import VBox, HBox, IntSlider, FloatSlider, Output, interactive_output, interactive

# Load the X-ray image (grayscale)
image_path = "xray_image.png"  # Replace with your actual image path
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

if image is None:
    raise FileNotFoundError(f"Image at path '{image_path}' could not be found or loaded.")
else:
    print("Image pixel values range:", image.min(), "to", image.max())

# Tissue categories for X-ray (you can adjust these ranges)
tissue_categories_xray = {
    "Bone": {"range": (200, 255), "color": [255, 0, 0], "color_name": "Red"},  # Bone, red
    "Soft Tissue": {"range": (80, 150), "color": [0, 255, 0], "color_name": "Green"},  # Soft tissue, green
    "Fat": {"range": (30, 80), "color": [0, 0, 255], "color_name": "Blue"},  # Fat, blue
    "Air": {"range": (0, 30), "color": [255, 255, 0], "color_name": "Yellow"},  # Air, yellow
}

# Function to map tissue types to colors for X-ray
def apply_tissue_colors_xray(image, tissue_map):
    colored_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    for tissue, info in tissue_map.items():
        lower, upper = info["range"]
        color = info["color"]
        mask = (image >= lower) & (image <= upper)
        for i in range(3):
            colored_image[:, :, i] = np.where(mask, color[i], colored_image[:, :, i])
    return colored_image

# Function to apply X-ray color filtering based on specific ranges
def apply_xray_color_filtering(image, ranges, colors):
    colored_image = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)
    for i, (lower, upper) in enumerate(ranges):
        color = colors[i]
        mask = (image >= lower) & (image <= upper)
        for j in range(3):
            colored_image[:, :, j] = np.where(mask, color[j], colored_image[:, :, j])
    return colored_image

# Function to display images side by side
def update_images_xray(
    bone_lower, bone_upper, soft_tissue_lower, soft_tissue_upper,
    fat_lower, fat_upper, air_lower, air_upper
):
    # Apply tissue color mapping to the X-ray image
    tissue_image = apply_tissue_colors_xray(image, tissue_categories_xray)

    # Define color ranges for X-ray tissue types (Bone, Soft Tissue, Fat, Air)
    ranges = [
        (bone_lower, bone_upper),
        (soft_tissue_lower, soft_tissue_upper),
        (fat_lower, fat_upper),
        (air_lower, air_upper),
    ]
    colors = [
        [255, 0, 0],  # Bone, red
        [0, 255, 0],  # Soft Tissue, green
        [0, 0, 255],  # Fat, blue
        [255, 255, 0],  # Air, yellow
    ]
    
    # Apply X-ray color filtering based on selected ranges
    filtered_xray_image = apply_xray_color_filtering(image, ranges, colors)

    # Display side-by-side images
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    
    # Grayscale Image
    axs[0].imshow(image, cmap='gray')
    axs[0].set_title(f'Grayscale X-ray')
    axs[0].axis('off')
    
    # Tissue Analysis
    axs[1].imshow(tissue_image)
    axs[1].set_title(f'X-ray Tissue Analysis')
    axs[1].axis('off')

    # Filtered X-ray Image
    axs[2].imshow(filtered_xray_image)
    axs[2].set_title(f'Filtered X-ray Image')
    axs[2].axis('off')

    plt.show()

    # Display tissue legend with tissue names and corresponding color names
    tissue_legend = "\n".join([f"{tissue} - {info['color_name']}" for tissue, info in tissue_categories_xray.items()])
    print("Tissue Color Mapping:\n", tissue_legend)

# X-ray sliders for tissue ranges
xray_sliders = VBox([
    IntSlider(min=0, max=255, step=1, value=200, description='Bone Range Lower'),
    IntSlider(min=0, max=255, step=1, value=255, description='Bone Range Upper'),
    IntSlider(min=0, max=255, step=1, value=80, description='Soft Tissue Range Lower'),
    IntSlider(min=0, max=255, step=1, value=150, description='Soft Tissue Range Upper'),
    IntSlider(min=0, max=255, step=1, value=30, description='Fat Range Lower'),
    IntSlider(min=0, max=255, step=1, value=80, description='Fat Range Upper'),
    IntSlider(min=0, max=255, step=1, value=0, description='Air Range Lower'),
    IntSlider(min=0, max=255, step=1, value=30, description='Air Range Upper'),
])

# Display the interactive controls for X-ray analysis
interactive_plot_xray = interactive_output(update_images_xray, {
    'bone_lower': xray_sliders.children[0], 'bone_upper': xray_sliders.children[1],
    'soft_tissue_lower': xray_sliders.children[2], 'soft_tissue_upper': xray_sliders.children[3],
    'fat_lower': xray_sliders.children[4], 'fat_upper': xray_sliders.children[5],
    'air_lower': xray_sliders.children[6], 'air_upper': xray_sliders.children[7],
})

display(VBox([xray_sliders, interactive_plot_xray]))


Image pixel values range: 0 to 255


In [ ]:
pip install opencv-python numpy matplotlib ipywidgets


  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.6 MB 11.0 MB/s eta 0:00:01
   -------------- ------------------------- 4.5/12.6 MB 11.3 MB/s eta 0:00:01
   ---------------------- ----------------- 7.1/12.6 MB 11.4 MB/s eta 0:00:01
   ------------------------------ --------- 9.4/12.6 MB 11.4 MB/s eta 0:00:01
   ------------------------------------- -- 11.8/12.6 MB 11.5 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 11.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ------------ -----